<div style="background-color:rgba(255, 0, 0, 0.5); padding: 10px; border-radius: 5px; width: 95%">

### **Limited autonomy of Large Language Models (LLMs)**

Large Language Models (LLMs) do not have real-time awareness or access to current events. Their knowledge is based on static training data with a fixed cut-off date.
They also cannot browse the web, run code, or interact with live tools unless explicitly connected to such systems.

As a result, when asked questions that require up-to-date information or external actions, LLMs may:

- Provide outdated, inaccurate, or nonsensical answers, **or**
- Admit they don’t know the answer.

**Example:**  
-  Ask the chatbot: _"What time is it?"_  
- and observe how it responds.

</div>

<div style="background-color:rgba(0, 0, 0, 0.5); padding: 10px; border-radius: 5px; width: 95%">

### **Purpose of the Notebook**
The goal of the notebook is to demonstrate how to overcome the limited autonomy of LLMs by connecting them to external tools and APIs, enabling them to access real-time information and perform actions beyond their static training data.

</div>

# Start Ollama Server

1. Open a terminal

    ☰ > *Terminal* > *New Terminal*

2. Start the Ollama server 

    ```bash
    ollama serve
    ```

3. Do not close this terminal

4. Open another terminal
5. Pull the model *"qwen3:4b"* (if not already done)

    ```bash
    ollama pull qwen3:4b
    ```
5. Close this terminal



# Define Environment Variables

In [4]:
OLLAMA_BASE_URL = "http://localhost:11434"
LLM_MODEL = "qwen3:4b"
LLM_SEED = 42 
LLM_TEMPERATURE = 0.0
TEST_PROMPT0 = "What time is it?"
TEST_PROMPT1 = "What is the price of gold right now?" 
TEST_PROMPT3 = "How many 1 in 111111111111111?"

# Initialize Ollama Chatbot

In [5]:
from langchain_ollama import ChatOllama

# Set up the Ollama chat model with specified LLM model and parameters
llm = ChatOllama(
    base_url=OLLAMA_BASE_URL,
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    seed=LLM_SEED,
    stream=True
)

# Define Tools

1. **Python Tool:** This tool allows the chatbot to execute Python code, enabling it to perform calculations, data processing, and other tasks that require programming logic.

2. **Commodity Price Tool:** This tool fetches real-time commodity prices from the `API Ninjas` API, allowing the chatbot to provide up-to-date information on various commodities.
    - Go to [``API Ninjas``](https://api-ninjas.com/)
    - Click the "Sign Up" button
    - Create an account
    - Log in
    - Click the "My Account" button
    - Click the "Show API Key" button

    In the next cell, you will need to:
    - Replace the content of the variable `NINJA_API_KEY` with your API key.
    - Run the cell to test the API tool

In [6]:
from langchain_core.tools import tool, Tool
from typing import Annotated, List
import io
import contextlib
import requests
import urllib3

NINJA_API_KEY = "MdaCcVUHseixkwpNGhumWg==9mb676ZThl4du9aP"

@tool
def execute_python(py_code: Annotated[str, "Python code to execute"]) -> str:
    """Executes a Python code and returns its standard output (you have to use the print() function)."""
    output = io.StringIO()
    try:
        with contextlib.redirect_stdout(output):
            exec(py_code, {})
        return output.getvalue().strip() or "Code executed with no output."
    except Exception as e:
        return f"Error: {str(e)}"

# Test the tool
print(execute_python.invoke("print('Hello, World!')"))

@tool
def get_commodity_price(
    commodity: Annotated[str, "The name of the commodity to get the price for"]
) -> str:
    """Get the current price of a commodity using the Ninja API."""
    api_url = 'https://api.api-ninjas.com/v1/commodityprice?name={}'.format(commodity)
    response = requests.get(api_url, headers={'X-Api-Key': NINJA_API_KEY})
    if response.status_code == requests.codes.ok:
        return response.text
    else:
        return f"Error {response.status_code}: {response.text}"

#Test the tool
print(get_commodity_price.invoke("gold"))
    
tools = [execute_python, get_commodity_price]




Hello, World!
{"exchange": "CME", "name": "Gold Futures", "price": 3368, "updated": 1749218320}


# Prepare Reason-and-Act (ReAct) Instructions

ReAct (Reason-and-Act) prompting combines:

- **Reasoning:** The model explains its thinking process.
- **Acting:** The model chooses and performs actions (like using a calculator, web search, or database query).

The prompt encourages the model to alternate between these two steps, creating a loop:

→ Think → Act → Observe → Think → Act → … until the task is complete.

**Reference:**

(Yao et al., 2023) Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2023, January). ReAct: Synergizing reasoning and acting in language models. In *International Conference on Learning Representations (ICLR)*.]{https://doi.org/10.48550/arXiv.2210.03629}

In [7]:
from langchain_core.runnables import RunnableLambda
from inspect import signature

tools_names = ", ".join([tool.name for tool in tools])
tools_descriptions = "\n".join([f"{tool.name}{signature(tool.func)} - {tool.description}" for tool in tools]) 


# Taken from https://smith.langchain.com/hub/hwchase17/react-json
instructions = f"""Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherwise, you have access to the following tools:

{tools_descriptions}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tools_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer:` when responding.
Always make your final answer easy to read and understand for humans.
"""

print(instructions)

Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherwise, you have access to the following tools:

execute_python(py_code: Annotated[str, 'Python code to execute']) -> str - Executes a Python code and returns its standard output (you have to use the print() function).
get_commodity_price(commodity: Annotated[str, 'The name of the commodity to get the price for']) -> str - Get the current price of a commodity using the Ninja API.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: execute_python, get_commodity_price

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

# Create a Chatbot Class

In [8]:
from typing import List
from langchain.schema import BaseMessage, AIMessage, HumanMessage, SystemMessage

class Chatbot:
    llm: ChatOllama
    history: List[BaseMessage]

    def __init__(self, llm: ChatOllama, history: List[BaseMessage] = []):
        """Initialize the chatbot with an LLM and an optional history."""
        self.llm = llm
        self.history = history
    
    def invoke(self, prompt:str) -> None:
        """Run the chatbot with the current history."""
        clear_output(wait=True)
        self.pretty_print()
        human_message = HumanMessage(content=prompt)
        human_message.pretty_print()
        self.history.append(human_message)
        ai_message = AIMessage(content="")
        ai_message.pretty_print()
        print()
        for chunk in self.llm.stream(self.history):
            print(chunk.content, end="")
            ai_message.content += chunk.content
        print()
        self.history.append(ai_message)
        
    
    def interact(self) -> None:
        """Run the chatbot in interactive mode."""
        while True: 
            prompt = input("Prompt (Enter 'stop' to exit)")
            if prompt == "stop": 
                break
            self.invoke(prompt)


    def pretty_print(self) -> None:
        """Pretty print the chatbot's history."""
        for message in self.history:
            message.pretty_print()

# Test the Chatbot

In [9]:
# Create a chat history with a system message and a human message
history = [SystemMessage(content=instructions)]
chatbot = Chatbot(llm=llm, history=history)
# chatbot.invoke(TEST_PROMPT0)
# chatbot.interact()

<div style= "padding: 0.5em;background-color: rgba(255,0,0, 0.5);width: 95%">

### **Problem:** the LLM generated the observation and should have waited for the tool response

Here, we did not interrupt the LLM when it generated the keyword *"Observation:"*. Meaning that we did not actually call the tool. Hence, the LLM hallucinated and made up a response.
</div>

# Interrupting the chatbot when generating '*Observations:*'

In [10]:

binded_llm = llm.bind(stop=["Observation:"])
history = [SystemMessage(content=instructions)]
chatbot = Chatbot(llm=binded_llm, history=history)
#chatbot.invoke(TEST_PROMPT0)
#chatbot.interact()

# Create an Agent Class

In [11]:
from langchain.schema import BaseMessage, AIMessage, HumanMessage, SystemMessage
import re
from typing import Union
from langchain_core.agents import AgentAction, AgentFinish, AgentActionMessageLog
from langchain_core.exceptions import OutputParserException
from langchain.agents.agent import AgentOutputParser
from langchain.agents.chat.prompt import FORMAT_INSTRUCTIONS
import ast
from IPython.display import clear_output, display
from langchain_core.messages import ToolMessage
import traceback
import sys

class Agent(Chatbot):

    tools: List[Tool]
    
    def __init__(self, llm: ChatOllama, tools: List[Tool], history: List[BaseMessage] = []):
        """Initialize the chatbot with an LLM, tools, and an optional history."""
        super().__init__(llm, history)
        self.tools = tools

    def invoke(self, prompt:str) -> None:
        """Run the chatbot in interactive mode."""
        self.history.append(HumanMessage(content=prompt))
        clear_output(wait=True)
        self.pretty_print()
        stop = False
        while not stop:
            ai_pre_action_message = self.llm.invoke(self.history)
            self.history.append(ai_pre_action_message)
            try:
                action = self.parse_action(ai_pre_action_message.content)
                if isinstance(action, AgentAction):
                    tool_message = self.call_tool(action)
                    self.history.append(tool_message)
                if isinstance(action, AgentFinish):
                    stop = True
            except SyntaxError as e:
                self.history.append(SystemMessage(content=str(e)))
            except Exception as e:
                traceback.print_exc()
                sys.exit()
            clear_output(wait=True)
            self.pretty_print()


    def call_tool(self, action: AgentAction) -> ToolMessage:
        """Call the specified tool with the given action input."""
        tool = next((t for t in self.tools if t.name == action.tool), None)
        if not tool:
            return ToolMessage(content=f"Error: Tool '{action.tool}' does not exist.", tool_call_id="unknown_tool")
        result = None
        sig = signature(tool.func)
        if len(sig.parameters):
            action_input = action.tool_input
            if not isinstance(action_input, dict):
                param_name = next(iter(sig.parameters))
                action_input = {param_name: action_input}
            result = tool.func(**action_input)
        else:
            result = tool.func()
        return ToolMessage(content=f"Observation: {result}", tool_call_id=tool.func.__name__)
    
    
    def parse_action(self, text:str) -> Union[AgentAction, AgentActionMessageLog, AgentFinish]:
        """Parse the action from the LLM output text and return an AgentAction or AgentFinish object."""
        FINAL_ANSWER_ACTION = "Final Answer:"
        pattern = re.compile(r"^.*?`{3}(?:json)?\n?(.*?)`{3}.*?$", re.DOTALL)
        includes_answer = FINAL_ANSWER_ACTION in text
        try:
            found = pattern.search(text)
            if not found:
                raise ValueError("action not found.")
            action = found.group(1)
            response = ast.literal_eval(action.strip())
            return AgentAction(response["action"], response.get("action_input", {}), text)
        except Exception as e:
            if not includes_answer:
                raise SyntaxError("Reminder to always use the exact characters `Final Answer:` when responding.")
            output = text.split(FINAL_ANSWER_ACTION)[-1].strip()
            return AgentFinish({"output": output}, text)


# Testing the Agent

In [13]:
# Create a chat history with a system message and a human message
binded_llm = llm.bind(stop=["Observation:"])
history = [SystemMessage(content=instructions)]
agent = Agent(llm=binded_llm, tools=tools, history=history)
agent.invoke(TEST_PROMPT0)
agent.invoke(TEST_PROMPT1)

================================ System Message ================================

Answer the following questions as best you can.
You can answer directly if the user is greeting you or similar.
Otherwise, you have access to the following tools:

execute_python(py_code: Annotated[str, 'Python code to execute']) -> str - Executes a Python code and returns its standard output (you have to use the print() function).
get_commodity_price(commodity: Annotated[str, 'The name of the commodity to get the price for']) -> str - Get the current price of a commodity using the Ninja API.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: execute_python, get_commodity_price

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of